In [ ]:
import numpy as np
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB4,EfficientNetB5,EfficientNetB6,EfficientNetB7
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [ ]:
from zipfile import ZipFile
with ZipFile('/content/New_Dataset.zip', 'r') as zipObj:

   zipObj.extractall('IMAGES')

In [ ]:
p = Augmentor.Pipeline("/content/IMAGES/images",output_directory="/content/output")

Initialised with 210 image(s) found.
Output directory set to /content/output.

In [ ]:
p.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)

p.zoom(probability=0.3, min_factor=1.1, max_factor=1.6)

In [ ]:
p.sample(1000)

Processing <PIL.Image.Image image mode=RGB size=416x416 at 0x78A3C06761A0>: 100%|██████████| 1000/1000 [00:38<00:00, 25.70 Samples/s]


In [ ]:
import pathlib
data_dir = pathlib.Path('/content/IMAGES/New_Dataset')
'''img_height=224
img_width=224'''

'img_height=224\nimg_width=224'

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(380, 380),
    batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(380, 380),
    batch_size=32
)


Found 1262 files belonging to 2 classes.
Using 1010 files for training.
Found 1262 files belonging to 2 classes.
Using 252 files for validation.


In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

In [ ]:
normalization_layer = Rescaling(1./255)
train_ds_normalized = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds_normalized = val_ds.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
num_classes=2
def one_hot_encoding(x, y):
    return x, tf.one_hot(y, depth=num_classes)

train_ds_encoded = train_ds_normalized.map(one_hot_encoding)
val_ds_encoded = val_ds_normalized.map(one_hot_encoding)

In [ ]:
train_ds_encoded = train_ds_encoded.prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds_encoded = val_ds_encoded.prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
model_filepath = "/content/drive/My Drive/model-{epoch:02d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(
    filepath=model_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

In [ ]:
base_model_b4 = EfficientNetB4(weights='imagenet', include_top=False, input_shape=(380, 380, 3))
base_model_b5 = EfficientNetB5(weights='imagenet', include_top=False, input_shape=(456, 456, 3))

115263384/115263384 [==============================] - 1s 0us/step


In [ ]:
from tensorflow.keras.layers import Dropout

In [ ]:
x_b4 = base_model_b4.output
x_b4 = GlobalAveragePooling2D()(x_b4)
x_b4 = Dense(1024, activation='relu')(x_b4)
x_b4 = Dropout(0.20)(x_b4)
x_b4 = Dense(512, activation='relu')(x_b4)
x_b4 = Dense(256, activation='relu')(x_b4)
x_b4 = Dense(64, activation='relu')(x_b4)

In [ ]:
output_b4 = Dense(num_classes, activation='softmax')(x_b4)
model_efficientnet_b4 = Model(inputs=base_model_b4.input, outputs=output_b4)
model_efficientnet_b4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
####EficientNetB5
x_b5 = base_model_b5.output
x_b5 = GlobalAveragePooling2D()(x_b5)
x_b5 = Dense(1024, activation='relu')(x_b5)
x_b5 = Dropout(0.20)(x_b5)
x_b5 = Dense(512, activation='relu')(x_b5)
#x_b5 = Dense(0.25)(x_b5)
x_b5 = Dense(256, activation='relu')(x_b5)
x_b5 = Dense(64, activation='relu')(x_b5)
x_b5 = Dense(16, activation='relu')(x_b5)
output_b5 = Dense(num_classes, activation='softmax')(x_b5)
model_efficientnet_b5 = Model(inputs=base_model_b5.input, outputs=output_b5)
model_efficientnet_b5.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model_efficientnet_b5.fit(
    train_ds_encoded,
    validation_data=val_ds_encoded,
    epochs=100,
    verbose=1
)

In [ ]:
test_label = np.concatenate([y for x, y in val_ds_normalized], axis=0)
print(test_label.shape)

(252,)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
Y_pred = model_efficientnet_b5.predict(val_ds_normalized)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
cm=confusion_matrix(test_label, y_pred)
print(cm)
print('Classification Report')
target_names = ['Pothole','SpeedBreaker']
print(classification_report(test_label, y_pred, target_names=target_names))

8/8 [==============================] - 2s 199ms/step
Confusion Matrix
[[  0 143]
 [  0 109]]
Classification Report
              precision    recall  f1-score   support

     Pothole       0.00      0.00      0.00       143
SpeedBreaker       0.43      1.00      0.60       109

    accuracy                           0.43       252
   macro avg       0.22      0.50      0.30       252
weighted avg       0.19      0.43      0.26       252



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Without Customization Accuracy was 0.61

In [ ]:
model_efficientnet_b4.fit(
    train_ds_encoded,
    validation_data=val_ds_encoded,
    epochs=25,
    verbose=1
)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
Y_pred = model_efficientnet_b4.predict(val_ds)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
cm=confusion_matrix(test_label, y_pred)
print(cm)
print('Classification Report')
target_names = ['Pothole','SpeedBreaker']
print(classification_report(test_label, y_pred, target_names=target_names))

In [ ]:
'''model_efficientnet_b6.fit(
    train_ds_encoded,
    validation_data=val_ds_encoded,
    epochs=100,
    verbose=1
)'''

In [ ]:
import pathlib
data_dir = pathlib.Path('/content/IMAGES/New_Dataset')

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(380, 380),  # Change the image size to match the new size
    batch_size=16
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(380, 380),  # Change the image size to match the new size
    batch_size=16
)

normalization_layer = Rescaling(1./255)
train_ds_normalized = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds_normalized = val_ds.map(lambda x, y: (normalization_layer(x), y))

num_classes = 2
def one_hot_encoding(x, y):
    return x, tf.one_hot(y, depth=num_classes)

train_ds_encoded = train_ds_normalized.map(one_hot_encoding)
val_ds_encoded = val_ds_normalized.map(one_hot_encoding)

#train_ds_encoded = train_ds_encoded.prefetch(buffer_size=tf.data.AUTOTUNE)
#val_ds_encoded = val_ds_encoded.prefetch(buffer_size=tf.data.AUTOTUNE)
train_ds_encoded = train_ds_encoded.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
val_ds_encoded = val_ds_encoded.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

base_model_b4 = EfficientNetB4(weights='imagenet', include_top=False, input_shape=(380, 380, 3))  # Change the input shape here

x_b4 = base_model_b4.output
x_b4 = GlobalAveragePooling2D()(x_b4)
#x_b4 = Dense(1024, activation='relu')(x_b4)
#x_b4 = Dropout(0.20)(x_b4)
x_b4 = Dense(512, activation='relu')(x_b4)
x_b4 = Dropout(0.20)(x_b4)
x_b4 = Dense(256, activation='relu')(x_b4)

output_b4 = Dense(num_classes, activation='softmax')(x_b4)  # Change the number of units here
model_efficientnet_b4 = Model(inputs=base_model_b4.input, outputs=output_b4)
model_efficientnet_b4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_efficientnet_b4.fit(
    train_ds_encoded,
    validation_data=val_ds_encoded,
    epochs=25,
    verbose=1
)
